In [17]:
import numpy as np
from digitalio import DigitalInOut, Direction
from adafruit_rgb_display import st7789
import board
from PIL import Image, ImageDraw, ImageFont, ImageOps
import time
import random
import cv2 as cv
import numpy as np
from colorsys import hsv_to_rgb
import os
import datetime

In [18]:
class Joystick:
    def __init__(self):
        self.cs_pin = DigitalInOut(board.CE0)
        self.dc_pin = DigitalInOut(board.D25)
        self.reset_pin = DigitalInOut(board.D24)
        self.BAUDRATE = 24000000

        self.spi = board.SPI()
        self.disp = st7789.ST7789(
                    self.spi,
                    height=240,
                    y_offset=80,
                    rotation=180,
                    cs=self.cs_pin,
                    dc=self.dc_pin,
                    rst=self.reset_pin,
                    baudrate=self.BAUDRATE,
                    )

        # Input pins:
        self.button_A = DigitalInOut(board.D5)
        self.button_A.direction = Direction.INPUT

        self.button_B = DigitalInOut(board.D6)
        self.button_B.direction = Direction.INPUT

        self.button_L = DigitalInOut(board.D27)
        self.button_L.direction = Direction.INPUT

        self.button_R = DigitalInOut(board.D23)
        self.button_R.direction = Direction.INPUT

        self.button_U = DigitalInOut(board.D17)
        self.button_U.direction = Direction.INPUT

        self.button_D = DigitalInOut(board.D22)
        self.button_D.direction = Direction.INPUT

        self.button_C = DigitalInOut(board.D4)
        self.button_C.direction = Direction.INPUT

        self.backlight = DigitalInOut(board.D26)
        self.backlight.switch_to_output()
        self.backlight.value = True

        self.width = self.disp.width
        self.height = self.disp.height

        # 배경 이미지 리스트 생성
        self.bg_images = ['bg/bg1.png', 'bg/bg2.png', 'bg/bg3.png']
        self.bg_images = [Image.open(bg).resize((self.width, self.height)) for bg in self.bg_images]
        self.bg_index = 0  # 현재 배경 이미지 인덱스


In [19]:
level1Fish = [
    "fish/f_1_1.png", "fish/f_1_2.png", "fish/f_1_3.png", "fish/f_1_4.png", "fish/f_1_5.png"]
level2Fish = [
    "fish/f_2_1.png", "fish/f_2_2.png", "fish/f_2_3.png", "fish/f_poison.png"]
level3Fish = [
    "fish/f_3_1.png", "fish/f_3_2.png", "fish/f_3_3.png", "fish/f_poison.png"]
#레벨 2, 3 에서만 독물고기 생성

class Fish:
    def __init__(self, level, width, height):
        self.level = level
        if level == 1:
            self.filename = random.choice(level1Fish)
        elif level == 2:
            self.filename = random.choice(level2Fish)
        elif level == 3:
            self.filename = random.choice(level3Fish)
        self.appearance = Image.open(self.filename)
        self.width = width
        self.height = height
        self.generate_random_fish()

        self.is_poisonous = self.filename.endswith("fish/f_poison.png")



    def generate_random_fish(self):
        fish_sizes = [(25, 25), (35, 35), (45, 45)]  # 레벨에 따른 물고기 크기
        base_image = self.appearance.resize(fish_sizes[self.level - 1])

        
        self.appearances = {
            'left': ImageOps.mirror(base_image),
            'right': base_image
        }
        self.appearance = self.appearances[random.choice(['left', 'right'])]
        x = random.randint(0, self.width - fish_sizes[self.level - 1][0])
        y = random.randint(0, self.height - fish_sizes[self.level - 1][1])
        self.position = np.array([x, y, x + fish_sizes[self.level - 1][0], y + fish_sizes[self.level - 1][1]])
        self.center = np.array([(self.position[0] + self.position[2]) / 2, (self.position[1] + self.position[3]) / 2])
        self.outline = "#FFFFFF"

    def move(self):
        if self.appearance == self.appearances['left']:
            self.position[0] -= 3
            self.position[2] -= 3
            if self.position[0] <= 0:
                self.appearance = self.appearances['right']
        else:
            self.position[0] += 3
            self.position[2] += 3
            if self.position[2] >= self.width:
                self.appearance = self.appearances['left']
        self.center = np.array([(self.position[0] + self.position[2]) / 2, (self.position[1] + self.position[3]) / 2])

    def get_score(self):
        # 레벨에 따른 점수 반환
        if self.is_poisonous:
            return -10  # 독성 물고기와 충돌하면 점수 감소
        else:
            if self.level == 1:
                return 10
            elif self.level == 2:
                return 30
            elif self.level == 3:
                return 50
    
    

In [20]:
class Antidote:
    def __init__(self, width, height):
        self.appearance = Image.open("etc/e_m.png")
        self.appearance = self.appearance.resize((20, 20))  # 이미지 크기 조정
        self.width = width
        self.height = height
        self.last_time = time.time()
        self.position = np.array([random.randint(20, self.width - 20), random.randint(20, self.height - 20)])  # 초기 위치 설정
        self.visible = True  # 해독제가 화면에 보이는지 여부
        self.appear()  # 객체 생성 시에도 위치를 생성합니다.

    def appear(self):
        if self.visible:  # 해독제가 화면에 보일 때만 위치 업데이트
            current_time = time.time()
            if current_time - self.last_time > 20:  # 20초마다 해독제 생성
                self.position = np.array([random.randint(20, self.width - 20), random.randint(20, self.height - 20)])  # 위치 업데이트
                self.last_time = current_time


In [21]:
class Mine:
    def __init__(self, width, height):
        self.appearance = Image.open("etc/e_mine.png")
        self.appearance = self.appearance.resize((50, 50))  # 이미지 크기 조정
        self.width = width
        self.height = height
        self.last_time = time.time()
        self.position = np.array([random.randint(50, self.width - 50), random.randint(50, self.height - 50)])  # 초기 위치 설정
        self.visible = True  # 마인이 화면에 보이는지 여부
        self.appear()  # 객체 생성 시에도 위치를 생성합니다.

    def appear(self):
        if self.visible:  # 마인이 화면에 보일 때만 위치 업데이트
            current_time = time.time()
            if current_time - self.last_time > 30:  # 30초마다 마인 생성
                self.position = np.array([random.randint(50, self.width - 50), random.randint(50, self.height - 50)])  # 위치 업데이트
                self.last_time = current_time

In [22]:
from PIL import Image, ImageOps

class Nemo:
    def __init__(self, width, height, joystick):
        self.level = 1
        self.load_images()
        self.set_level(self.level)  # 이 부분에서 이미지 로딩 및 크기 조절이 이루어집니다.
        self.joystick=joystick
        

        self.state = None
        self.position = np.array([width/2 - 20, height/2 - 20, width/2 + 20, height/2 + 20])
        self.center = np.array([(self.position[0] + self.position[2]) / 2, (self.position[1] + self.position[3]) / 2])
        self.outline = "#FFFFFF"
        self.joystick = joystick

        self.antidote_count = 0
        self.is_poisoned = False

    def load_images(self):
        # 모든 레벨에 대한 이미지를 미리 로드하고 크기를 조절합니다.
        self.all_images = {}
        for level in range(1, 4):
            if level == 1: size = 50
            elif level == 2: size = 65
            else: size = 80

            base_image = Image.open('fish/f_nemo.png').resize((size, size))
            self.all_images[level] = {
                'up': base_image.rotate(270),
                'down': base_image.rotate(90),
                'right': ImageOps.mirror(base_image.rotate(0)),
                'left': base_image.rotate(0)  
            }

            #
    def set_level(self, level):
        self.level = level
        self.appearances = self.all_images[level]  # 레벨에 맞는 이미지를 선택합니다.
        self.appearance = self.appearances['left']

    def move(self, command=None):
        if command['move'] == False:
            self.state = None
            self.outline = "#FFFFFF"  # 검정색상 코드!
        else:
            self.state = 'move'
            self.outline = "#FF0000"  # 빨강색상 코드!

            if command['up_pressed']:
                if (self.position[1] > 0) or (self.position[1] <= 0 and self.joystick.bg_index > 0):  # bg1에서는 위로는 더 이동할 수 없음
                    self.position[1] -= 5
                    self.position[3] -= 5
                    self.appearance = self.appearances['up']  # 위쪽으로 움직일 때 이미지 변경

            if command['down_pressed']:
                if (self.position[3] < self.joystick.height) or (self.position[3] >= self.joystick.height and self.joystick.bg_index < len(self.joystick.bg_images) - 1):  # bg3에서는 아래로는 더 이동할 수 없음
                    self.position[1] += 5
                    self.position[3] += 5
                    self.appearance = self.appearances['down']  # 아래쪽으로 움직일 때 이미지 변경

            if command['left_pressed']:
                if self.position[0] > 0:  # 화면 왼쪽 끝에 도달하지 않았을 때만 이동 가능
                    self.position[0] -= 5
                    self.position[2] -= 5
                    self.appearance = self.appearances['left']  # 왼쪽으로 움직일 때 이미지 변경

            if command['right_pressed']:
                if self.position[2] < self.joystick.width:  # 화면 오른쪽 끝에 도달하지 않았을 때만 이동 가능
                    self.position[0] += 5
                    self.position[2] += 5
                    self.appearance = self.appearances['right']  # 오른쪽으로 움직일 때 이미지 변경
            
            if self.is_poisoned:
                if self.antidote_count > 0 and self.joystick.button_B.value == False:  # B 버튼을 누르면 해독제 사용
                    self.antidote_count -= 1
                    self.is_poisoned = False
                
        #center update
        self.center = np.array([(self.position[0] + self.position[2]) / 2, (self.position[1] + self.position[3]) / 2]) 


In [23]:
def draw_game_screen(my_image, my_Nemo, joystick, fish_list, antidote, mine, score):
    draw = ImageDraw.Draw(my_image)

    my_image.paste(change_background(my_Nemo, joystick, fish_list))  
    my_image.paste(my_Nemo.appearance, (int(my_Nemo.position[0]), int(my_Nemo.position[1])), my_Nemo.appearance)

    for fish in fish_list:
        my_image.paste(fish.appearance, (int(fish.position[0]), int(fish.position[1])), fish.appearance)

    my_image.paste(antidote.appearance, (antidote.position[0], antidote.position[1]))  # 해독제 그리기
    #my_image.paste(mine.appearance, (mine.position[0], mine.position[1]))  # 마인 그리기
    my_image.paste(mine.appearance, (mine.position[0], mine.position[1]), mine.appearance)


    
    font_path = "/usr/share/fonts/truetype/dejavu/DejaVuSans-Bold.ttf"  # 폰트 경로 설정
    font = ImageFont.truetype(font_path, 15)  # 폰트 타입과 크기 설정

    antidote_text = f"Antidote: {my_Nemo.antidote_count}"
    draw.text((10, 30), antidote_text, font=font, fill="black")  # 해독제 개수 출력

    
    score_text = f"Score: {score}"
    draw.text((10, 10), score_text, font=font, fill="black")  # 스코어 텍스트를 직접 이미지에 출력

    joystick.disp.image(my_image)



def clear_fish(joystick, fish_list):
    fish_list.clear()  # 기존 물고기 리스트를 비움


def generate_fish(joystick, fish_list):
    if joystick.bg_index == 0:  # bg1에서는 레벨 1의 물고기만 생성
        fish_list.append(Fish(1, joystick.width, joystick.height))
    elif joystick.bg_index == 1:  # bg2에서는 레벨 2의 물고기만 생성
        fish_list.append(Fish(2, joystick.width, joystick.height))
    elif joystick.bg_index == 2:  # bg3에서는 레벨 3의 물고기만 생성
        fish_list.append(Fish(3, joystick.width, joystick.height))


def change_background(my_Nemo, joystick, fish_list):
    if my_Nemo.position[3] >= joystick.height and joystick.bg_index < len(joystick.bg_images) - 1:  
        joystick.bg_index += 1  
        my_Nemo.position[1] = 0  
        my_Nemo.position[3] = my_Nemo.appearance.height  
        clear_fish(joystick, fish_list)
        generate_fish(joystick, fish_list)
    elif my_Nemo.position[1] <= 0 and joystick.bg_index > 0:  
        joystick.bg_index -= 1  
        my_Nemo.position[1] = joystick.height - my_Nemo.appearance.height  
        my_Nemo.position[3] = joystick.height  
        clear_fish(joystick, fish_list)
        generate_fish(joystick, fish_list)

    return joystick.bg_images[joystick.bg_index]


def calculate_distance(point1, point2):
    distance = np.sqrt(np.sum((point1 - point2)**2))
    #print(f"Distance: {distance}, Point1: {point1}, Point2: {point2}")  # 로그 출력
    return distance


def eat_fish(nemo, fish):
    distance = calculate_distance(nemo.center, fish.center)
    if distance < 20:
        if nemo.level < fish.level:  # 니모의 레벨이 물고기의 레벨보다 낮은 경우
            return -50  # 점수를 50점 깎음
        else:
            return fish.get_score()  # 그렇지 않으면 물고기의 점수를 반환
    else:
        return 0  # 물고기와 부딪히지 않은 경우
    

# 기록 저장 함수
def save_record(clear_time):
    with open('game_records.txt', 'a') as f:
        f.write(f"{clear_time.total_seconds()}\n")

# 기록 불러오기 함수
def load_records():
    if not os.path.exists('game_records.txt'):
        return []

    with open('game_records.txt', 'r') as f:
        records = f.readlines()

    records = [float(record.strip()) for record in records if record.strip()]
    top_three_records = sorted(records)[:3]  # 시간이 짧은 순으로 정렬
    return [round(record, 2) for record in top_three_records]  # 초 단위 시간을 반올림하여 반환



In [24]:
def main():
    joystick = Joystick()
    my_image = Image.new("RGB", (joystick.width, joystick.height))
    my_draw = ImageDraw.Draw(my_image)

    # 시작 화면 이미지 로드
    start_image = Image.open('bg/bg_start.png').resize((joystick.width, joystick.height))



    # 클리어 화면 이미지 생성
    clear_image = Image.new("RGB", (joystick.width, joystick.height), "white")
    clear_draw = ImageDraw.Draw(clear_image)
    font = ImageFont.truetype('/usr/share/fonts/truetype/dejavu/DejaVuSans-Bold.ttf', size=45)  # 글꼴과 크기 설정
    text = "CLEAR!!!"
    text_width, text_height = clear_draw.textsize(text, font=font)
    position = ((joystick.width-text_width)/2, (joystick.height-text_height)/2)
    clear_draw.text(position, text, font=font, fill="black")

    fail_image = Image.new("RGB", (joystick.width, joystick.height), "white")
    fail_draw = ImageDraw.Draw(fail_image)
    font = ImageFont.truetype('/usr/share/fonts/truetype/dejavu/DejaVuSans-Bold.ttf', size=45)  # 글꼴과 크기 설정
    text = "FAIL!!!"
    text_width, text_height = fail_draw.textsize(text, font=font)
    position = ((joystick.width-text_width)/2, (joystick.height-text_height)/2)
    fail_draw.text(position, text, font=font, fill="black")

    while True:  # 사용자가 A 버튼을 누르면 게임이 시작되고, 게임이 종료되면 이 루프의 처음으로 돌아감
        
        top_three_records = load_records()
        start_image = Image.open('bg/bg_start.png').resize((joystick.width, joystick.height))
        draw = ImageDraw.Draw(start_image)
        font_path = "/usr/share/fonts/truetype/dejavu/DejaVuSans-Bold.ttf"
        font = ImageFont.truetype(font_path, 15)
        j=len(top_three_records)
        for i, record in enumerate(reversed(top_three_records), 1):
            record_text = f"{j}: {record}"
            text_width, text_height = font.getsize(record_text)
            position = (joystick.width - text_width - 10, joystick.height - text_height - 10 - i*20)
            draw.text(position, record_text, font=font, fill="white")
            j = j - 1

        my_image.paste(start_image, (0, 0))  # 시작 화면 표시
        joystick.disp.image(my_image)

        while True:  # 사용자가 A 버튼을 누를 때까지 대기
            if joystick.button_A.value == False:  
                break
            time.sleep(0.1)  # CPU 사용 줄이기 위한 sleep

        nemoLevel=1
        my_Nemo = Nemo(joystick.width, joystick.height, joystick)
        fish_list = []
        antidote = Antidote(joystick.width, joystick.height)
        last_fish_time = time.time()
        score=0
        poison_start_time = None

        my_mine = Mine(joystick.width, joystick.height)  # 마인 객체 생성

        start_time = datetime.datetime.now()
        
        while True:  # 게임 로직
            #start_time = datetime.datetime.now()
            command = {'move': False, 'up_pressed': False , 'down_pressed': False, 'left_pressed': False, 'right_pressed': False}
        
            if not joystick.button_U.value:  
                command['up_pressed'] = True
                command['move'] = True

            if not joystick.button_D.value:  
                command['down_pressed'] = True
                command['move'] = True

            if not joystick.button_L.value:  
                command['left_pressed'] = True
                command['move'] = True

            if not joystick.button_R.value:  
                command['right_pressed'] = True
                command['move'] = True

            my_Nemo.move(command)

            current_time = time.time()
            if current_time - last_fish_time > 5:  
                generate_fish(joystick, fish_list)  # 배경에 맞는 물고기 생성
                last_fish_time = current_time

            antidote.appear()  # 해독제 생성

            distance_to_antidote = calculate_distance(my_Nemo.center, antidote.position)
            if distance_to_antidote < 20:  # 해독제를 주울 수 있는 거리에 있는 경우
                my_Nemo.antidote_count += 1
                antidote.visible = False  # 해독제를 먹었으므로 해독제를 화면에서 숨김
                antidote.position = np.array([-100, -100])  # 해독제를 화면 밖으로 이동

            if current_time - antidote.last_time > 20:  # 20초가 지났으면
                antidote.visible = True  # 해독제를 다시 화면에 보이게 함
                antidote.appear()  # 해독제 위치 업데이트

            
            my_mine.appear()  # 마인 생성

            # 마인과 니모의 거리 계산 후, 마인에 닿았을 경우 점수 초기화 및 레벨 돌리기
            distance_to_mine = calculate_distance(my_Nemo.center, my_mine.position)
            if distance_to_mine < 25:  # 마인에 닿았을 경우
                score = 0
                my_Nemo.set_level(1)
                my_mine.visible = False  # 마인을 먹었으므로 마인을 화면에서 숨김
                my_mine.position = np.array([-100, -100])  # 마인을 화면 밖으로 이동

            if current_time - my_mine.last_time > 30:  # 30초가 지났으면
                my_mine.visible = True  # 마인을 다시 화면에 보이게 함
                my_mine.appear()  # 마인 위치 업데이트


            i = 0
            while i < len(fish_list):
                fish = fish_list[i]
                fish.move()
                eat_score = eat_fish(my_Nemo, fish)
                if eat_score != 0:
                    score += eat_score
                    del fish_list[i]
                    if score > 300:
                        my_Nemo.set_level(3)
                    elif score > 100:
                        my_Nemo.set_level(2)
                    if fish.is_poisonous:
                        my_Nemo.is_poisoned = True
                        poison_start_time = current_time
                else:
                    i += 1

            if my_Nemo.is_poisoned:
                if current_time - poison_start_time >= 2:  # 독성 물고기를 먹은 후 2초가 지난 경우
                    score -= 10
                    poison_start_time = current_time
                if joystick.button_B.value == False:  # B 버튼을 누른 경우
                    if my_Nemo.antidote_count > 0:  # 해독제를 사용할 수 있는 경우
                        my_Nemo.is_poisoned = False
                        my_Nemo.antidote_count -= 1

            draw_game_screen(my_image, my_Nemo, joystick, fish_list, antidote, my_mine, score)  # 해독제 그리기 추가
            
            if score >= 1000:  # 점수가 1000점 이상인 경우
                end_time = datetime.datetime.now()  # 클리어 시간 기록
                clear_time = end_time - start_time  # 클리어 시간 계산 (여기서는 문자열로 변환하지 않습니다.)
                save_record(clear_time)  # 클리어 시간을 기록
                joystick.disp.image(clear_image)  # 클리어 화면 표시
                time.sleep(3)  # 3초 기다림
                break  # 게임 로직 종료, 게임 시작 대기 루프로 돌아감

            if score <= -10:  # 점수가 -10점 이하인 경우
                joystick.disp.image(fail_image)  # 실패 화면 표시
                time.sleep(3)  # 3초 기다림
                break  # 게임 로직 종료, 게임 시작 대기 루프로 돌아감

if __name__ == '__main__':
    main()


KeyboardInterrupt: 